<a href="https://colab.research.google.com/github/Isaacgtz17/Embebed/blob/master/Kommo_Proyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
#         CUADERNO DE GOOGLE COLAB PARA ANÁLISIS DE VENTAS DE KOMMO
# =============================================================================
#
# VERSIÓN PROFESIONAL CON GENERACIÓN DE REPORTES EN PDF
#
# Instrucciones:
# 1. Abre Google Colab: https://colab.research.google.com/
# 2. Crea un nuevo cuaderno (notebook).
# 3. GENERA TU TOKEN: En Kommo, ve a Ajustes -> Integraciones, crea o edita tu
#    integración y genera un "Long-lived access token". Cópialo.
# 4. GUARDA EL TOKEN: En Colab, haz clic en el ícono de la llave (🔑) en el panel
#    izquierdo ("Secretos"). Agrega los siguientes secretos:
#    - KOMMO_ACCESS_TOKEN: Pega aquí el token de larga duración que copiaste.
#    - KOMMO_SUBDOMAIN: Tu subdominio (ej. "tueempresa")
# 5. Copia y pega el código de esta celda en tu notebook y ejecútalo.
# =============================================================================

# =============================================================================
# PASO 1: INSTALACIÓN Y CONFIGURACIÓN DEL ENTORNO
# =============================================================================
print("--- PASO 1: INSTALACIÓN Y CONFIGURACIÓN DEL ENTORNO ---")

# Instalar las librerías necesarias
!pip install requests pandas matplotlib seaborn fpdf2 -q

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import matplotlib.ticker as mticker
from fpdf import FPDF
import io
import numpy as np

# Importar la librería para acceder a los secretos de Colab
from google.colab import userdata

# --- Carga de Credenciales ---
try:
    SUBDOMAIN = userdata.get('KOMMO_SUBDOMAIN')
    ACCESS_TOKEN = userdata.get('KOMMO_ACCESS_TOKEN')
    print("Credenciales (subdominio y token) cargadas correctamente desde los secretos.")
    if not SUBDOMAIN or not ACCESS_TOKEN:
        raise ValueError("El subdominio o el token están vacíos. Revisa los secretos en Colab.")
except Exception as e:
    print(f"Error al cargar las credenciales: {e}")
    print("Por favor, asegúrate de haber configurado los secretos 'KOMMO_SUBDOMAIN' y 'KOMMO_ACCESS_TOKEN' en Google Colab.")
    ACCESS_TOKEN = None

# Configuración de estilo para las gráficas
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14

print("--- Fin del Paso 1 ---")


# =============================================================================
# PASO 2: CONEXIÓN Y EXTRACCIÓN DE DATOS DE LA API DE KOMMO
# =============================================================================
print("\n--- PASO 2: CONEXIÓN Y EXTRACCIÓN DE DATOS ---")

if SUBDOMAIN:
    BASE_URL = f"https://{SUBDOMAIN}.kommo.com"
else:
    BASE_URL = None
    print("Error: No se puede definir BASE_URL porque el SUBDOMAIN no se cargó.")

# --- Funciones de la API ---
def get_api_data(endpoint):
    if not ACCESS_TOKEN or not BASE_URL: return None
    url, headers = f"{BASE_URL}{endpoint}", {'Authorization': f'Bearer {ACCESS_TOKEN}'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de {endpoint}: {e}")
        return None

def get_users_map():
    data = get_api_data('/api/v4/users')
    return {user['id']: user['name'] for user in data.get('_embedded', {}).get('users', [])} if data else {}

def get_pipelines_and_statuses_map():
    data = get_api_data('/api/v4/leads/pipelines')
    pipelines_map, statuses_map = {}, {}
    if data and '_embedded' in data:
        for pipeline in data['_embedded']['pipelines']:
            pipeline_statuses = [{'id': s['id'], 'name': s['name']} for s in sorted(pipeline['_embedded']['statuses'], key=lambda x: x['sort'])]
            pipelines_map[pipeline['id']] = {'name': pipeline['name'], 'statuses': pipeline_statuses}
            for status in pipeline['_embedded']['statuses']:
                statuses_map[status['id']] = {'name': status['name'], 'pipeline': pipeline['name']}
    return pipelines_map, statuses_map

def get_all_leads():
    if not ACCESS_TOKEN: return []
    all_leads, page = [], 1
    print("Iniciando descarga de leads...")
    while True:
        data = get_api_data(f'/api/v4/leads?limit=250&page={page}&with=contacts,companies')
        if data and '_embedded' in data and 'leads' in data['_embedded']:
            leads_on_page = data['_embedded']['leads']
            all_leads.extend(leads_on_page)
            print(f"  - Página {page} descargada. Total de leads: {len(all_leads)}")
            if len(leads_on_page) < 250: break
            page += 1
            time.sleep(0.5)
        else:
            break
    print(f"Descarga completa. Se encontraron {len(all_leads)} leads en total.")
    return all_leads

# --- Ejecución de la extracción ---
if ACCESS_TOKEN:
    users_map = get_users_map()
    pipelines_map, statuses_map = get_pipelines_and_statuses_map()
    all_leads_data = get_all_leads()
else:
    all_leads_data = []

print("--- Fin del Paso 2 ---")


# =============================================================================
# PASO 3: PROCESAMIENTO Y TRANSFORMACIÓN DE DATOS (DATA WAREHOUSING)
# =============================================================================
print("\n--- PASO 3: PROCESAMIENTO Y TRANSFORMACIÓN DE DATOS ---")

def procesar_datos(leads_data):
    if not leads_data: return None
    df = pd.DataFrame(leads_data)
    print("DataFrame inicial creado. Mapeando y limpiando datos...")

    # Mapeos básicos
    df['responsable_nombre'] = df['responsible_user_id'].map(users_map)
    df['creado_por_nombre'] = df['created_by'].map(users_map)
    df['pipeline_nombre'] = df['pipeline_id'].map(lambda x: pipelines_map.get(x, {}).get('name'))
    df['etapa_nombre'] = df['status_id'].map(lambda x: statuses_map.get(x, {}).get('name'))

    # Procesamiento de fechas y renombrado
    df['fecha_creacion'] = pd.to_datetime(df['created_at'], unit='s', utc=True).dt.tz_localize(None)
    df['fecha_actualizacion'] = pd.to_datetime(df['updated_at'], unit='s', utc=True).dt.tz_localize(None)

    # Filtrar datos futuros para asegurar que el análisis sea solo sobre el pasado
    df = df[df['fecha_creacion'] <= datetime.now()]

    df.rename(columns={'name': 'nombre_lead', 'price': 'valor'}, inplace=True)

    columnas_interes = ['id', 'nombre_lead', 'valor', 'fecha_creacion', 'fecha_actualizacion',
                        'responsable_nombre', 'creado_por_nombre',
                        'pipeline_nombre', 'etapa_nombre', 'status_id', 'pipeline_id']

    print("Procesamiento de datos finalizado.")
    return df.reindex(columns=columnas_interes).copy()

df_final = procesar_datos(all_leads_data) if all_leads_data else pd.DataFrame()

print("--- Fin del Paso 3 ---")


# =============================================================================
# PASO 4: FUNCIONES DE ANÁLISIS Y REPORTES
# =============================================================================

def crear_reporte_pdf(titulo_reporte, kpis, graficas_info):
    """Crea y guarda un reporte profesional en PDF."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font('Helvetica', 'B', 24)
    pdf.cell(0, 20, 'Reporte de Analítica de Ventas', 0, 1, 'C')
    pdf.set_font('Helvetica', 'B', 16)
    pdf.cell(0, 10, titulo_reporte, 0, 1, 'C')
    pdf.set_font('Helvetica', '', 11)
    pdf.cell(0, 10, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 0, 1, 'C')
    pdf.ln(10)

    if kpis:
        pdf.set_font('Helvetica', 'B', 12)
        pdf.cell(0, 10, "Indicadores Clave de Rendimiento (KPIs)", 0, 1, 'L')
        pdf.set_font('Helvetica', '', 11)
        for k, v in kpis.items():
            pdf.cell(0, 8, f"- {k}: {v}", 0, 1, 'L')

    for info in graficas_info:
        pdf.add_page()
        pdf.set_font('Helvetica', 'B', 14)
        pdf.cell(0, 10, info['titulo'], 0, 1, 'L')
        pdf.set_font('Helvetica', '', 10)
        pdf.multi_cell(0, 5, info['descripcion'], 0, 'L')
        pdf.ln(5)
        pdf.image(info['buffer'], x=10, y=None, w=pdf.w - 20)

    nombre_archivo_pdf = f"Reporte_Kommo_{titulo_reporte.replace(' ', '_').replace('/', '-')}.pdf"
    pdf.output(nombre_archivo_pdf)
    print(f"Reporte en PDF guardado como: {nombre_archivo_pdf}")

def generar_reporte_detallado(df_reporte, titulo_principal):
    """Genera un reporte detallado con múltiples gráficas y lo guarda en PDF."""
    print(f"\n--- GENERANDO REPORTE DETALLADO: {titulo_principal} ---")
    if df_reporte.empty:
        print("No se encontraron leads en el periodo seleccionado para generar el reporte.")
        return

    graficas_para_pdf = []
    ETAPA_GANADO_ID, ETAPA_PERDIDO_ID = 142, 143

    # --- Cálculo de KPIs ---
    total_leads = len(df_reporte)
    leads_ganados = len(df_reporte[df_reporte['status_id'] == ETAPA_GANADO_ID])
    valor_total_ganado = pd.to_numeric(df_reporte[df_reporte['status_id'] == ETAPA_GANADO_ID]['valor'], errors='coerce').sum()
    tasa_conversion = (leads_ganados / total_leads * 100) if total_leads > 0 else 0
    kpis = {
        "Total de Leads Generados": f"{total_leads}",
        "Total de Leads Ganados": f"{leads_ganados}",
        "Tasa de Conversión General": f"{tasa_conversion:.2f}%",
        "Valor Total de Ventas Ganadas": f"${valor_total_ganado:,.2f}",
    }

    # --- Gráfica 1: Evolución de Leads (General) ---
    print("   - Generando gráfica: Evolución de Leads...")
    df_reporte = df_reporte.sort_values('fecha_creacion')
    agrupacion_tiempo = 'M' if (df_reporte['fecha_creacion'].max() - df_reporte['fecha_creacion'].min()).days > 90 else 'W'
    agrupacion = df_reporte.resample(agrupacion_tiempo, on='fecha_creacion')['id'].count()

    fig, ax = plt.subplots()
    agrupacion.plot(ax=ax, marker='.', linestyle='-')
    titulo_grafica = f'Evolución de Leads Creados ({ "Mensual" if agrupacion_tiempo == "M" else "Semanal"})'
    ax.set_title(titulo_grafica)
    ax.set_ylabel('Cantidad de Leads')
    ax.set_xlabel('Fecha')
    plt.tight_layout()
    img_buffer = io.BytesIO(); plt.savefig(img_buffer, format='png'); plt.close(fig)
    graficas_para_pdf.append({'titulo': titulo_grafica, 'buffer': img_buffer, 'descripcion': 'Muestra la cantidad de nuevos leads a lo largo del tiempo para identificar tendencias.'})

    # --- Gráfica 2: Distribución de Responsabilidad (Histórico) ---
    print("   - Generando gráfica: Distribución de Responsabilidad...")
    responsables_df = df_reporte.groupby('responsable_nombre')['id'].count().sort_values(ascending=False)

    fig, ax = plt.subplots(figsize=(12, 8))
    sns.barplot(x=responsables_df.values, y=responsables_df.index, ax=ax, palette='coolwarm', orient='h')
    titulo_grafica = 'Distribución de Leads por Responsable (Total Histórico)'
    ax.set_title(titulo_grafica)
    ax.set_xlabel('Cantidad Total de Leads Asignados')
    ax.set_ylabel('Ejecutivo')
    for i, v in enumerate(responsables_df.values):
        ax.text(v, i, f" {v}", va='center')
    plt.tight_layout()
    img_buffer = io.BytesIO(); plt.savefig(img_buffer, format='png'); plt.close(fig)
    graficas_para_pdf.append({'titulo': titulo_grafica, 'buffer': img_buffer, 'descripcion': 'Muestra el total de leads que cada ejecutivo ha gestionado históricamente. Ayuda a entender la carga de trabajo y quiénes son los principales responsables de la cartera.'})

    # --- Gráfica 3: Análisis por cada Embudo de Ventas ---
    print("   - Generando análisis por cada embudo de ventas...")
    for pipeline_id, pipeline_info in pipelines_map.items():
        nombre_embudo = pipeline_info['name']
        df_embudo = df_reporte[df_reporte['pipeline_id'] == pipeline_id]
        if df_embudo.empty: continue

        etapas_ordenadas = [s['name'] for s in pipeline_info['statuses'] if s['id'] not in [ETAPA_GANADO_ID, ETAPA_PERDIDO_ID]]
        conteo_etapas = df_embudo['etapa_nombre'].value_counts()
        funnel_df = pd.DataFrame({'etapa': etapas_ordenadas, 'cantidad': [conteo_etapas.get(e, 0) for e in etapas_ordenadas]})
        funnel_df['tasa_conversion'] = 100 * (funnel_df['cantidad'] / funnel_df['cantidad'].shift(1)).fillna(1)

        fig, ax = plt.subplots(figsize=(12, 8))
        sns.barplot(x='cantidad', y='etapa', data=funnel_df, orient='h', ax=ax, palette='viridis')
        titulo_grafica = f"Análisis del Embudo - {nombre_embudo}"
        ax.set_title(titulo_grafica)
        for index, row in funnel_df.iterrows():
            label = f" {row['cantidad']:.0f} Leads" + (f" ({row['tasa_conversion']:.1f}%)" if index > 0 else "")
            ax.text(row['cantidad'], index, label, color='black', ha="left", va="center")
        plt.tight_layout()
        img_buffer = io.BytesIO(); plt.savefig(img_buffer, format='png'); plt.close(fig)
        graficas_para_pdf.append({'titulo': titulo_grafica, 'buffer': img_buffer, 'descripcion': f'Visualiza cuántos leads pasan de una etapa a la siguiente en el embudo "{nombre_embudo}", identificando cuellos de botella.'})

    # --- Gráfica 4: Tabla de KPIs por Ejecutivo ---
    print("   - Generando Tabla de Rendimiento por Ejecutivo...")
    if not df_reporte['responsable_nombre'].isnull().all():
        kpis_ejecutivos = df_reporte.groupby('responsable_nombre').agg(
            leads_asignados=('id', 'count'),
            leads_ganados=('status_id', lambda x: (x == ETAPA_GANADO_ID).sum()),
            valor_ventas=('valor', lambda x: pd.to_numeric(x, errors='coerce').sum())
        ).reset_index()
        kpis_ejecutivos['tasa_conversion_%'] = (kpis_ejecutivos['leads_ganados'] / kpis_ejecutivos['leads_asignados'] * 100).fillna(0)
        kpis_ejecutivos = kpis_ejecutivos.sort_values('leads_ganados', ascending=False)

        fig, ax = plt.subplots(figsize=(12, max(2, len(kpis_ejecutivos)*0.5)))
        ax.axis('off')
        titulo_grafica = "Tabla de Rendimiento por Ejecutivo"
        ax.set_title(titulo_grafica, pad=20)

        kpis_ejecutivos['valor_ventas'] = kpis_ejecutivos['valor_ventas'].apply(lambda x: f"${x:,.0f}")
        kpis_ejecutivos['tasa_conversion_%'] = kpis_ejecutivos['tasa_conversion_%'].apply(lambda x: f"{x:.1f}%")

        tabla = ax.table(cellText=kpis_ejecutivos.values, colLabels=kpis_ejecutivos.columns, loc='center', cellLoc='center')
        tabla.auto_set_font_size(False); tabla.set_fontsize(10); tabla.scale(1.2, 1.2)

        img_buffer = io.BytesIO(); plt.savefig(img_buffer, format='png', bbox_inches='tight'); plt.close(fig)
        graficas_para_pdf.append({'titulo': titulo_grafica, 'buffer': img_buffer, 'descripcion': "Resume el rendimiento de cada ejecutivo, comparando cantidad de leads, ventas logradas, tasa de conversión y valor total generado."})

    print("Finalizando la generación de gráficas...")
    crear_reporte_pdf(titulo_principal, kpis, graficas_para_pdf)

def analizar_rango_fechas(df):
    """Función para analizar un único rango de fechas y generar un PDF detallado."""
    print("\n--- ANÁLISIS POR RANGO DE FECHAS ---")
    try:
        fecha_inicio_str = input("Fecha de Inicio (AAAA-MM-DD): ")
        fecha_fin_str = input("Fecha de Fin (AAAA-MM-DD): ")
        fecha_inicio = pd.to_datetime(fecha_inicio_str)
        fecha_fin = pd.to_datetime(fecha_fin_str)
    except (ValueError, TypeError):
        print("Entrada de fecha no válida. Volviendo al menú principal.")
        return

    df_filtrado = df[(df['fecha_creacion'] >= fecha_inicio) & (df['fecha_creacion'] < fecha_fin + timedelta(days=1))]
    titulo = f"Reporte del {fecha_inicio_str} al {fecha_fin_str}"

    generar_reporte_detallado(df_filtrado, titulo)

def comparar_rangos_fechas(df):
    """Función para comparar dos rangos de fechas y generar un PDF."""
    print("\n--- COMPARATIVA DE PERIODOS ---")
    try:
        print("\n-- Periodo A --")
        inicio_a_str, fin_a_str = input("Fecha de Inicio A (AAAA-MM-DD): "), input("Fecha de Fin A (AAAA-MM-DD): ")
        print("\n-- Periodo B --")
        inicio_b_str, fin_b_str = input("Fecha de Inicio B (AAAA-MM-DD): "), input("Fecha de Fin B (AAAA-MM-DD): ")
        inicio_a, fin_a = pd.to_datetime(inicio_a_str), pd.to_datetime(fin_a_str)
        inicio_b, fin_b = pd.to_datetime(inicio_b_str), pd.to_datetime(fin_b_str)
    except (ValueError, TypeError):
        print("Entrada de fecha no válida. Volviendo al menú principal.")
        return

    df_a = df[(df['fecha_creacion'] >= inicio_a) & (df['fecha_creacion'] < fin_a + timedelta(days=1))]
    df_b = df[(df['fecha_creacion'] >= inicio_b) & (df['fecha_creacion'] < fin_b + timedelta(days=1))]

    def calcular_kpis(dfp):
        total_leads = len(dfp)
        if total_leads == 0: return [0, 0, '0.0%', '$0']
        leads_ganados = len(dfp[dfp['status_id'] == 142])
        tasa_conv = (leads_ganados / total_leads * 100)
        valor_ganado = pd.to_numeric(dfp[dfp['status_id'] == 142]['valor'], errors='coerce').sum()
        return [total_leads, leads_ganados, f"{tasa_conv:.1f}%", f"${valor_ganado:,.0f}"]

    df_kpi = pd.DataFrame({
        'Métrica': ['Total de Leads', 'Leads Ganados', 'Tasa de Conversión', 'Valor Total Ganado'],
        f"Periodo A ({inicio_a_str} a {fin_a_str})": calcular_kpis(df_a),
        f"Periodo B ({inicio_b_str} a {fin_b_str})": calcular_kpis(df_b),
    })

    fig, ax = plt.subplots(figsize=(10, 3))
    ax.axis('off')
    tabla = ax.table(cellText=df_kpi.values, colLabels=df_kpi.columns, loc='center', cellLoc='center', colWidths=[0.4, 0.3, 0.3])
    tabla.auto_set_font_size(False); tabla.set_fontsize(10); tabla.scale(1, 2)
    plt.tight_layout()
    img_buffer_tabla = io.BytesIO()
    plt.savefig(img_buffer_tabla, format='png', bbox_inches='tight')
    plt.close(fig)

    crear_reporte_pdf(f"Comparativa A vs B", None, [{'titulo': "Tabla Comparativa de KPIs", 'buffer': img_buffer_tabla, 'descripcion': "Comparación directa de los indicadores de rendimiento clave entre los dos periodos."}])


# =============================================================================
# PASO 5: MENÚ PRINCIPAL INTERACTIVO
# =============================================================================

def menu_principal():
    """Muestra el menú principal y maneja la selección del usuario."""
    if df_final.empty:
        print("\nNo se pudieron cargar los datos. El programa no puede continuar.")
        return

    # 1. Generar y guardar un reporte PDF detallado con todos los datos históricos al iniciar.
    print("\nGenerando el reporte histórico general. Por favor, espere...")
    generar_reporte_detallado(df_final, "Reporte Histórico General")

    while True:
        print("\n\n--- MENÚ PRINCIPAL DE ANÁLISIS ---")
        print("Seleccione la opción que desea ejecutar:")
        print("1. Analizar un Rango de Fechas Específico")
        print("2. Comparar dos Periodos")
        print("3. Salir")

        opcion = input("Introduzca su opción (1-3): ")

        if opcion == '1':
            analizar_rango_fechas(df_final)
        elif opcion == '2':
            comparar_rangos_fechas(df_final)
        elif opcion == '3':
            print("Saliendo del programa de análisis. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, intente de nuevo.")

if __name__ == '__main__':
    menu_principal()


--- PASO 1: INSTALACIÓN Y CONFIGURACIÓN DEL ENTORNO ---
Credenciales (subdominio y token) cargadas correctamente desde los secretos.
--- Fin del Paso 1 ---

--- PASO 2: CONEXIÓN Y EXTRACCIÓN DE DATOS ---
Iniciando descarga de leads...
  - Página 1 descargada. Total de leads: 250
  - Página 2 descargada. Total de leads: 254
Descarga completa. Se encontraron 254 leads en total.
--- Fin del Paso 2 ---

--- PASO 3: PROCESAMIENTO Y TRANSFORMACIÓN DE DATOS ---
DataFrame inicial creado. Mapeando y limpiando datos...
Procesamiento de datos finalizado.
--- Fin del Paso 3 ---

Generando el reporte histórico general. Por favor, espere...

--- GENERANDO REPORTE DETALLADO: Reporte Histórico General ---
   - Generando gráfica: Evolución de Leads...


/tmp/ipython-input-7-155472933.py:229: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  agrupacion = df_reporte.resample(agrupacion_tiempo, on='fecha_creacion')['id'].count()


   - Generando gráfica: Distribución de Responsabilidad...
   - Generando análisis por cada embudo de ventas...


/tmp/ipython-input-7-155472933.py:246: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=responsables_df.values, y=responsables_df.index, ax=ax, palette='coolwarm', orient='h')
/tmp/ipython-input-7-155472933.py:270: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='cantidad', y='etapa', data=funnel_df, orient='h', ax=ax, palette='viridis')
/tmp/ipython-input-7-155472933.py:270: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='cantidad', y='etapa', data=funnel_df, orient='h', ax=ax, palette='viridis')
/tmp/ipython-input-7-155472933.py:270: Futur

   - Generando Tabla de Rendimiento por Ejecutivo...
Finalizando la generación de gráficas...


/tmp/ipython-input-7-155472933.py:176: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 20, 'Reporte de Analítica de Ventas', 0, 1, 'C')
/tmp/ipython-input-7-155472933.py:178: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, titulo_reporte, 0, 1, 'C')
/tmp/ipython-input-7-155472933.py:180: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 0, 1, 'C')
/tmp/ipython-input-7-155472933.py:185: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Indicadores Clave de Rendimiento (KPIs)", 0, 1, 'L')
/tmp/ipython-input-7-155472933.py:188: DeprecationWarning: The parameter "l

Reporte en PDF guardado como: Reporte_Kommo_Reporte_Histórico_General.pdf


--- MENÚ PRINCIPAL DE ANÁLISIS ---
Seleccione la opción que desea ejecutar:
1. Analizar un Rango de Fechas Específico
2. Comparar dos Periodos
3. Salir
